In [318]:
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from catboost import CatBoostClassifier
import xgboost
from xgboost import  XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

import pandas as pd
import numpy as np

tf.get_logger().setLevel('ERROR')
tf.test.is_gpu_available()


2024-03-02 20:28:41.204694: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-02 20:28:41.204784: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-02 20:28:41.204799: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-02 20:28:41.205003: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-02 20:28:41.205017: I tensorflow/core/common_runtime/gpu/gpu

True

87] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-02 20:28:41.205063: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /device:GPU:0 with 5592 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:06:00.0, compute capability: 8.6


In [319]:
apple_data = pd.read_csv("apple_processed_data.csv", index_col=0)
tesla_data = pd.read_csv("tesla_processed_data.csv", index_col=0)

apple_data['month'] = apple_data['month'].astype('category')
apple_data['day'] = apple_data['day'].astype('category')

tesla_data['month'] = tesla_data['month'].astype('category')
tesla_data['day'] = tesla_data['day'].astype('category')

In [320]:
def train_valid_test(data, target, split):
    train, valid, test = split
    n = data.shape[0]
    train_end = int(train * n)
    valid_end = int(n * (train + valid))
    x_train, y_train = data[:train_end], target[:train_end]
    x_val, y_val = data[train_end:valid_end], target[train_end:valid_end]
    x_test, y_test = data[valid_end:], target[valid_end:]

    return x_train, y_train, x_val, y_val, x_test, y_test

In [321]:
data = apple_data
data.head()

,Date,close,open,lowest,highest,total_vol,mean_vol,std_vol,news,month,day,is_up
0,2015-01-02,24.490,24.925,24.026,24.992,188181988,482517.917949,453958.983288,"[""Is Xiaomi's first laptop a MacBook Air knock...",1,2,0
1,2015-01-05,23.807,24.192,23.592,24.366,200586492,514324.338462,426711.446836,"[""Is Xiaomi's first laptop a MacBook Air knock...",1,5,0
2,2015-01-06,23.802,23.842,23.417,24.092,237766160,609656.820513,452106.984355,"[""Is Xiaomi's first laptop a MacBook Air knock...",1,6,1
3,2015-01-07,24.143,24.017,23.879,24.265,137809632,353358.030769,315345.332594,Apple aims at bendable devices\nA newly issued...,1,7,1
4,2015-01-08,25.066,24.391,24.328,25.151,201020076,515436.092308,344928.788130,"Xiaomi's buying spree gives Apple, Samsung mor...",1,8,1


In [322]:
predictors = data.drop(['is_up', 'Date', 'news', 'day', 'month', 'total_vol', 'mean_vol', 'std_vol'], axis=1)
target = data['is_up']

predictors['ema5'] = predictors['close'].ewm(span=5).mean()
for i in range(1, 8):
    predictors['lag_' + str(i)] = predictors['close'].shift(i)
predictors = predictors.iloc[7:]
predictors

,close,open,lowest,highest,ema5,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
7,24.697,24.958,24.375,25.297,24.636187,24.481,25.095,25.066,24.143,23.802,23.807,24.490
8,24.600,24.468,24.283,24.779,24.623803,24.697,24.481,25.095,25.066,24.143,23.802,23.807
9,23.936,24.521,23.871,24.682,24.390489,24.600,24.697,24.481,25.095,25.066,24.143,23.802
10,23.746,23.990,23.545,24.126,24.173147,23.936,24.600,24.697,24.481,25.095,25.066,24.143
11,24.358,24.051,23.836,24.395,24.235243,23.746,23.936,24.600,24.697,24.481,25.095,25.066
...,...,...,...,...,...,...,...,...,...,...,...,...
2218,171.090,171.810,170.650,173.060,173.174476,173.390,172.960,172.840,175.450,175.840,177.140,178.675
2219,166.760,170.325,165.670,171.377,171.036317,171.090,173.390,172.960,172.840,175.450,175.840,177.140
2220,168.285,167.110,166.830,168.960,170.119212,166.760,171.090,173.390,172.960,172.840,175.450,175.840
2221,170.230,169.160,168.870,171.170,170.156141,168.285,166.760,171.090,173.390,172.960,172.840,175.450


In [323]:
x_train, y_train, x_val, y_val, x_test, y_test = train_valid_test(predictors, target, (0.6, 0.2, 0.2))

## Transform data

In [324]:
sc = StandardScaler()
sc.fit(x_train)
x_train_scaled = sc.transform(x_train)
x_val_scaled = sc.transform(x_val)
x_test_scaled = sc.transform(x_test)

## Baseline model
According to EDA, the market was up ~ 52% of the days, so the simplest model would be always predict up, which could potentially achieve 52% accuracy.

In [325]:
counting = data['is_up'].value_counts()
BASELINE = counting[1] / np.sum(counting)
BASELINE

0.5263157894736842

In [326]:
accuracy_score(y_val, [1] * len(y_val))

0.5440180586907449

The baseline model has accuracy of ~ **54%**

## Default hyperparameters with some Algo

Logistic regression

In [327]:
lr = LogisticRegression(random_state = 12)
lr.fit(x_train_scaled, y_train)

print("Training accuracy:", lr.score(x_train_scaled, y_train))
print("Validation accuracy:", lr.score(x_val_scaled, y_val))

Training accuracy: 0.8284424379232506
Validation accuracy: 0.8194130925507901


Random Forest Classifier

In [328]:
rf = RandomForestClassifier(bootstrap=True, random_state = 7)
rf.fit(x_train_scaled, y_train)

print("Training accuracy:", rf.score(x_train_scaled, y_train))
print("Validation accuracy:", rf.score(x_val_scaled, y_val))

Training accuracy: 1.0
Validation accuracy: 0.5507900677200903


In [329]:
sorted(list(zip(predictors.columns, rf.feature_importances_)), key= lambda x: -x[1])

[('lag_7', 0.1788118512299655),
 ('lag_6', 0.1125848090486385),
 ('lag_5', 0.09106816712921678),
 ('lag_2', 0.07585625319896798),
 ('lag_4', 0.07481224317546097),
 ('lag_3', 0.07402469786313687),
 ('ema5', 0.07098761181164438),
 ('lag_1', 0.06875185402126728),
 ('close', 0.06531133018745403),
 ('open', 0.06526522230276896),
 ('highest', 0.06307675468776391),
 ('lowest', 0.05944920534371484)]

AdaBoost

In [330]:
ada = AdaBoostClassifier(random_state = 12)
ada.fit(x_train_scaled, y_train)

print("Training accuracy:", ada.score(x_train_scaled, y_train))
print("Validation accuracy:", ada.score(x_val_scaled, y_val))

Training accuracy: 0.7057938299473289
Validation accuracy: 0.4672686230248307


CatBoostClassifier

In [331]:
cat = CatBoostClassifier(silent=True, random_state = 12)
cat.fit(x_train_scaled, y_train)

print("Training accuracy:", cat.score(x_train_scaled, y_train))
print("Validation accuracy:", cat.score(x_val_scaled, y_val))

Training accuracy: 0.8954100827689992
Validation accuracy: 0.5485327313769752


XGBoost Classifier 

In [332]:
xg = XGBClassifier()
xg.fit(x_train_scaled, y_train)

print("Training accuracy:", xg.score(x_train_scaled, y_train))
print("Validation accuracy:", xg.score(x_val_scaled, y_val))

Training accuracy: 0.9984951091045899
Validation accuracy: 0.47404063205417607


Simple Deep Learning

In [333]:
x_train_lstm = np.reshape(x_train_scaled, (x_train_scaled.shape[0], x_train_scaled.shape[1], 1))

model = tf.keras.Sequential()
unit = 16
model.add(LSTM(unit, return_sequences=False, input_shape=(x_train_lstm.shape[1], 1)))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'tanh'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [334]:
hist = model.fit(x_train_scaled, y_train, epochs=15)

Epoch 1/15
42/42 [==============================] - 1s 5ms/step - loss: 3.8611 - accuracy: 0.4891
Epoch 2/15
42/42 [==============================] - 0s 5ms/step - loss: 1.4986 - accuracy: 0.5004
Epoch 3/15
42/42 [==============================] - 0s 5ms/step - loss: 0.7310 - accuracy: 0.5184
Epoch 4/15
42/42 [==============================] - 0s 4ms/step - loss: 0.7096 - accuracy: 0.5320
Epoch 5/15
42/42 [==============================] - 0s 5ms/step - loss: 0.7070 - accuracy: 0.5214
Epoch 6/15
42/42 [==============================] - 0s 5ms/step - loss: 0.7166 - accuracy: 0.5117
Epoch 7/15
42/42 [==============================] - 0s 5ms/step - loss: 0.7250 - accuracy: 0.5162
Epoch 8/15
42/42 [==============================] - 0s 5ms/step - loss: 0.7089 - accuracy: 0.5124
Epoch 9/15
42/42 [==============================] - 0s 5ms/step - loss: 0.7115 - accuracy: 0.5192
Epoch 10/15
42/42 [==============================] - 0s 5ms/step - loss: 0.7352 - accuracy: 0.5094
Epoch 11/15
42/42 [

In [335]:
threshold = 0
y_val_pred = (model.predict(x_val_scaled) > threshold).astype(int)
print("Training accuracy:", accuracy_score(y_train, (model.predict(x_train_scaled) > threshold).astype(int)))
print("Validation accuracy:", accuracy_score(y_val, y_val_pred))

42/42 [==============================] - 0s 2ms/step
Training accuracy: 0.5259593679458239
Validation accuracy: 0.5440180586907449


## Optimize hyper params

In [336]:
import optuna
from optuna import Trial

n_trials = 30
random_state = 7

### Logis Regression

In [337]:
def lr_optimize(trial: Trial):
    solver = trial.suggest_categorical("solver", ["newton-cholesky", "newton-cholesky", "lbfgs"])
    penalty = trial.suggest_categorical("penalty", ['l2', None])
    lr = LogisticRegression(penalty=penalty, solver=solver, random_state=random_state)
    lr.fit(x_train_scaled, y_train)
    y_val_pred = lr.predict(x_val_scaled)
    return accuracy_score(y_val, y_val_pred)

In [338]:
lr_study = optuna.create_study(direction="maximize")
lr_study.optimize(lr_optimize, n_trials=n_trials)

[I 2024-03-02 20:28:50,336] A new study created in memory with name: no-name-97e90326-07da-4f13-a526-5b36b1e30544
[I 2024-03-02 20:28:50,343] Trial 0 finished with value: 0.8194130925507901 and parameters: {'solver': 'newton-cholesky', 'penalty': 'l2'}. Best is trial 0 with value: 0.8194130925507901.
[I 2024-03-02 20:28:50,358] Trial 1 finished with value: 0.9909706546275395 and parameters: {'solver': 'lbfgs', 'penalty': None}. Best is trial 1 with value: 0.9909706546275395.
[I 2024-03-02 20:28:50,370] Trial 2 finished with value: 0.9909706546275395 and parameters: {'solver': 'lbfgs', 'penalty': None}. Best is trial 1 with value: 0.9909706546275395.
/home/vinhbui/miniconda3/envs/neural-net/lib/python3.10/site-packages/sklearn/linear_model/_glm/_newton_solver.py:498: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #10. It will now resort to lbfgs instead.
Further options are to use another solver or to 

In [339]:
print("Best params", lr_study.best_params)
print("Best accuracy", lr_study.best_value)

Best params {'solver': 'newton-cholesky', 'penalty': None}
Best accuracy 0.9954853273137697


In [340]:
np.vstack([x_train_scaled, x_val_scaled]).shape

(1772, 12)

In [341]:
def lstm_optimize(trial: Trial):
    pass